In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd, numpy as np
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import urbansim.sim.simulation as sim

In [2]:
sim.run(["build_networks"]) #initialize network accessibility engine
sim.run(["scheduled_development_events", "neighborhood_vars", #scheduled events and accessibility variables
         "rsh_simulate", "nrh_simulate", "nrh_simulate2",   #price models
         "price_vars", "feasibility", "residential_developer", "non_residential_developer", #supply/proforma models
         "buildings_to_uc"
         ], years=[2013,])

Running model 'build_networks'
Time to execute model 'build_networks': 13.94s
Total time to execute: 13.94s
Running year 2013
Running model 'scheduled_development_events'
Time to execute model 'scheduled_development_events': 0.73s
Running model 'neighborhood_vars'
Computing accessibility variables
Computing ave_parcel_size
Computing mean_nonres_rent_2000m
Computing jobs_1500m
Computing jobs_800m
Computing jobs_400m
Computing jobs1
Computing jobs2
Computing jobs3
Computing jobs4
Computing jobs5
Computing jobs6
Computing jobs7
Computing jobs8
Computing jobs9
Computing jobs10
Computing jobs11
Computing jobs12
Computing jobs13
Computing jobs14
Computing jobs15
Computing jobs16
Computing jobs17
Computing jobs18
Computing jobs19
Computing jobs20
Computing ave_income
Computing ave_age_of_head_1500m
Computing ave_children_1500m
Computing ave_year_built_1500m
Computing population_400m
Computing job_spaces_3000m
Computing jobs_3000m
Computing households_3000m
Computing residential_units_3000m
Co

c:\Anaconda\lib\site-packages\pandas\util\decorators.py:81: FutureWarning: the 'cols' keyword is deprecated, use 'subset' instead
  warnings.warn(msg, FutureWarning)
c:\Anaconda\lib\site-packages\pandas\computation\expressions.py:190: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  unsupported[op_str]))



Unplaced jobs after: 2934
Simulated DU: 47263.0
Target DU: 46006
Time to execute model 'residential_developer': 7.45s
Running model 'non_residential_developer'
768,606 feasible buildings before running developer
Job space target for heavy_industrial is -29036.5317706.
Job space target for light_industrial is -155958.16065.
Job space target for retail is 103824.295.
Sum of net units that are profitable: 401,264
Constructed 3706 retail buildings, totaling 105957.307715 new job spaces.
Unplaced households before: 3240
Unplaced households after: 8998
Unplaced jobs before: 2934
Unplaced jobs after: 3800
Job space target for office is 5387.16581.
Sum of net units that are profitable: 69,399
Constructed 287 office buildings, totaling 5478.59147096 new job spaces.
Unplaced households before: 8998
Unplaced households after: 9422
Unplaced jobs before: 3800
Unplaced jobs after: 3846
Time to execute model 'non_residential_developer': 5.29s
Running model 'buildings_to_uc'
Exporting 7618 buildings 

c:\Anaconda\lib\site-packages\pandas\io\sql.py:1511: FutureWarning: read_frame is deprecated, use read_sql
  warnings.warn("read_frame is deprecated, use read_sql", FutureWarning)


In [34]:
import psycopg2
import pandas.io.sql as sql
import cStringIO

In [3]:
buildings = sim.get_table('buildings')
b = buildings.to_frame(buildings.local_columns)
year = sim.get_injectable('year')

new_buildings =  b[(b.note=='simulated') | (b.note.str.startswith('Sitespec'))]
new_buildings = new_buildings[new_buildings.year_built == year]

In [9]:
new_buildings[new_buildings.stories>2].stories.describe()

count    134.000000
mean       3.186567
std        0.564194
min        3.000000
25%        3.000000
50%        3.000000
75%        3.000000
max        5.000000
Name: stories, dtype: float64

In [10]:
new_buildings.columns

Index([u'parcel_id', u'development_type_id', u'improvement_value', u'residential_units', u'non_residential_sqft', u'stories', u'year_built', u'residential_sqft', u'note', u'res_price_per_sqft', u'nonres_rent_per_sqft'], dtype='object')

In [20]:
new_buildings = new_buildings.reset_index()
new_buildings = new_buildings.rename(columns = {'development_type_id':'building_type_id'})
new_buildings['building_sqft'] = new_buildings.residential_sqft + new_buildings.non_residential_sqft
new_buildings['sqft_per_unit'] =  new_buildings.residential_sqft/new_buildings.residential_units

In [22]:
del new_buildings['res_price_per_sqft']
del new_buildings['nonres_rent_per_sqft']

In [25]:
# Urban Canvas database connection
if 'uc_conn' not in sim._INJECTABLES.keys():
    conn_string = "host='urbancanvas.cp2xwchuariu.us-west-2.rds.amazonaws.com' dbname='sandag_testing' user='sandag' password='parcel22building' port=5432"
    conn=psycopg2.connect(conn_string)
    cur = conn.cursor()
    
    sim.add_injectable('uc_conn', conn)
    sim.add_injectable('uc_cur', cur)
    
else:
    conn = sim.get_injectable('uc_conn')
    cur = sim.get_injectable('uc_cur')
    

In [26]:
max_bid = sql.read_frame("select max(building_id) FROM building where building_id<100000000;",conn)  #should this be lowered?
max_bid = max_bid.values[0][0]
new_buildings.building_id = np.arange(max_bid+1, max_bid+1+len(new_buildings))

c:\Anaconda\lib\site-packages\pandas\io\sql.py:1511: FutureWarning: read_frame is deprecated, use read_sql
  warnings.warn("read_frame is deprecated, use read_sql", FutureWarning)


In [31]:
def exec_sql_uc(query):
    cur.execute(query)
    conn.commit()

if 'projects_num' not in sim._INJECTABLES.keys(): 
    nextval = sql.read_frame("SELECT MAX(ID)+1 FROM SCENARIO WHERE ID < 100000;",conn)
    nextval = nextval.values[0][0]
    sim.add_injectable('projects_num', nextval)
    
    id = sql.read_frame("select max(id)+1 from scenario_project;",conn)
    id = id.values[0][0]
    
    exec_sql_uc("INSERT INTO scenario(id, name) VALUES(%s, 'Run #%s');" % (nextval,nextval))
    
    exec_sql_uc("INSERT INTO scenario_project(id, scenario, project) VALUES(%s, %s, 1);" % (id,nextval))
                
    id = sql.read_frame("select max(id)+1 from scenario_project;",conn)
    id = id.values[0][0]
    
    exec_sql_uc("INSERT INTO scenario_project(id, scenario, project) VALUES(%s, %s, %s);" % (id,nextval,nextval))
    
    
else:
    nextval = sim.get_injectable('projects_num')

In [44]:
new_buildings.parcel_id = new_buildings.parcel_id.astype('int32')
new_buildings.residential_units = new_buildings.residential_units.astype('int32')
new_buildings.non_residential_sqft = new_buildings.non_residential_sqft.astype('int32')
new_buildings.stories = new_buildings.stories.astype('int32')
new_buildings.residential_sqft = new_buildings.residential_sqft.astype('int32')
new_buildings.building_sqft = new_buildings.building_sqft.fillna(0).astype('int32')
new_buildings.sqft_per_unit = new_buildings.sqft_per_unit.fillna(0).astype('int32')

In [45]:
conn_string = "host='urbancanvas.cp2xwchuariu.us-west-2.rds.amazonaws.com' dbname='sandag_testing' user='sandag' password='parcel22building' port=5432"
conn=psycopg2.connect(conn_string)
cur = conn.cursor()

In [46]:
new_buildings.dtypes

building_id              int64
parcel_id                int32
building_type_id         int64
improvement_value        int64
residential_units        int32
non_residential_sqft     int32
stories                  int32
year_built               int64
residential_sqft         int32
note                    object
building_sqft            int32
sqft_per_unit            int32
projects                object
valid_from              object
dtype: object

In [47]:
nextval = '{' + str(nextval) + '}'
new_buildings['projects'] = nextval

valid_from = '{' + str(year) + '-1-1}'
new_buildings['valid_from'] = valid_from
print 'Exporting %s buildings to Urbancanvas database for project %s and year %s.' % (len(new_buildings),nextval,year)
output = cStringIO.StringIO()
new_buildings.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
cur.copy_from(output, 'building', columns =tuple(new_buildings.columns.values.tolist()))
conn.commit()

Exporting 7809 buildings to Urbancanvas database for project {{{{11}}}} and year 2013.
